In [1]:
# Package Imports
from os import makedirs
from pandas import merge, read_parquet, to_datetime

In [2]:
# Variables
dest_folder = 'midp-307'
ticket_2_agg = {'email': 'first',
                'quantity': 'sum',
                'last_name': 'first',
                'first_name': 'first'}

sales_loc = 'final_data/sales.parquet'
customer_loc = 'final_data/customer.parquet'

report_1_loc = f'{dest_folder}/sales_report.csv'
report_2_loc = f'{dest_folder}/sales_report_active_member.csv'

required_cols = ['customer_id', 'first_name', 'last_name', 'email',
                 'quantity', 'effective_from', 'membership_status']

In [3]:
# Data Extract
sales_data = read_parquet(sales_loc)
customer_data = read_parquet(customer_loc)

In [4]:
# Create Ticket Folder
makedirs(dest_folder, exist_ok=True)

In [5]:
## Query 1
# Data Cleaning
customer_wise = sales_data.groupby('customer_id').quantity.sum().reset_index()

merged_data = merge(customer_wise, customer_data,
                    on='customer_id', how='inner')[required_cols]

final_data = merged_data.loc[merged_data.groupby('customer_id')\
                                        .effective_from.idxmax()]\
                        .sort_values('quantity', ascending=False)\
                        [:10][required_cols[1:5]]

# Data Load
final_data.to_csv(report_1_loc, index=False)

In [6]:
## Query 2
# Data Cleaning
sales_data.sale_date = to_datetime(sales_data.sale_date)
active_cust = customer_data[customer_data.membership_status == 'active']

merged_data = merge(sales_data, active_cust, on='customer_id',
                    how='inner', suffixes=['', '_cust'])
merged_data = merged_data[merged_data.sale_date\
                                     .between(merged_data.effective_from_cust,
                                              merged_data.expiry_date_cust)]\
                         .drop_duplicates('sale_id')

final_data = merged_data.groupby('customer_id').agg(ticket_2_agg)\
                        .reset_index().sort_values('quantity',
                                                   ascending=False)\
                        [:10][required_cols[1:5]]

# Data Load
final_data.to_csv(report_2_loc, index=False)

In [8]:
customer_data[customer_data.email == 'lisajones@example.org']

,is_current,expiry_date,effective_from,customer_id,hash,first_name,last_name,gender,email,membership_status,address,phone_number,date_of_birth,job,company,city,state,country,language
2488,False,2024-10-10,2024-10-09,416636781440851074,9140524628082735876,Kathryn,Patterson,M,lisajones@example.org,active,"02583 Nicole Trail\nAustinshire, OR 29054",None,1976-12-18,"Surveyor, insurance",Sandoval-Davis,Jamesstad,None,Tanzania,en-US
5071,True,2099-12-31,2024-10-10,416636781440851074,1547775518071631588,Kathryn,Campbell,M,lisajones@example.org,active,"02583 Nicole Trail\nAustinshire, OR 29054",None,1976-12-18,"Surveyor, insurance",Sandoval-Davis,Jamesstad,None,Tanzania,en-US
